In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import re

In [2]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\112025\realty_sold_09112025_M.xlsx")


In [3]:
df2 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\112025\realty_sold_09112025_MO.xlsx")

In [4]:
df3 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\112025\realty_sold_09112025_NM.xlsx")

In [5]:
df_общий = pd.concat([df, df2, df3], ignore_index=True)

In [6]:
df_общий.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2153074 entries, 0 to 2153073
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ID ЖК          int64  
 1   ЖК рус         object 
 2   Тип помещения  object 
 3   Корпус         object 
 4   ID Корпус      float64
 5   lat            float64
 6   lng            float64
 7   ID дом.рф      float64
dtypes: float64(4), int64(1), object(3)
memory usage: 131.4+ MB


In [7]:
df_фильтр = df_общий[(df_общий['Тип помещения'] == 'квартира') | (df_общий['Тип помещения'] == 'апартамент')]

In [8]:
df_фильтр = df_фильтр[df_фильтр['ID дом.рф'].notna() & (df_фильтр['ID дом.рф'] != "")]

In [9]:
df_фильтр = df_фильтр.drop_duplicates()

In [44]:
df_фильтр.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5981 entries, 3 to 2152510
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID ЖК          5981 non-null   int64  
 1   ЖК рус         5981 non-null   object 
 2   Тип помещения  5981 non-null   object 
 3   Корпус         5981 non-null   object 
 4   ID Корпус      5981 non-null   float64
 5   lat            5981 non-null   float64
 6   lng            5981 non-null   float64
 7   ID дом.рф      5981 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 420.5+ KB


In [68]:
df_фильтр

,ID ЖК,ЖК рус,Тип помещения,Корпус,ID Корпус,lat,lng,ID дом.рф
3,1603,1147,квартира,1,5419.0,55.812770,37.647610,22163.0
3455,1940,20-20,квартира,Б,7447.0,55.713240,37.596220,23271.0
3456,1940,20-20,квартира,А,7446.0,55.713240,37.596220,23271.0
3788,8539,А22,квартира,1,255.0,55.705498,37.635181,64921.0
3793,4303,Аалто,квартира,1,8484.0,55.801420,37.561420,1551.0
...,...,...,...,...,...,...,...,...
2151766,4223,Эко Бунино,квартира,12,10476.0,55.541420,37.455760,35081.0
2152270,6659,Юнино,квартира,2.2,18521.0,55.514366,37.574926,58056.0
2152274,6659,Юнино,квартира,2.1,18520.0,55.514366,37.574926,58056.0
2152509,6659,Юнино,квартира,1.1,17467.0,55.514366,37.574926,49819.0


In [50]:
df_фильтр['Корпус'] = df_фильтр['Корпус'].str.replace('корпус ', '', regex=False)

In [17]:
df_фильтр['Кол-во уникальных ID'] = (
    df_фильтр.groupby('ID ЖК')['ID дом.рф'].transform('nunique')
)

In [20]:
df_фильтр = df_фильтр[df_фильтр["Кол-во уникальных ID"] == 1]

In [55]:
df_baze = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База изменяемые данные.xlsx")

In [56]:
df_baze2 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База проектов.xlsx")

In [60]:
df_baze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360 entries, 0 to 2359
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Название проекта                2359 non-null   object 
 1   Девелопер                       2360 non-null   object 
 2   Корпус                          2359 non-null   object 
 3   Срок сдачи                      2359 non-null   object 
 4   Стадия строительной готовности  2359 non-null   object 
 5   Договор                         2359 non-null   object 
 6   id                              2199 non-null   float64
dtypes: float64(1), object(6)
memory usage: 129.2+ KB


In [58]:
df_merged = df_baze.merge(
    df_фильтр[['ID ЖК', 'ID дом.рф']],
    left_on='id',
    right_on='ID ЖК',
    how='left'
)

KeyError: 'id'

In [59]:
df_baze = df_baze.merge(
    df_baze2[['Название проекта', 'id']],
    on='Название проекта',
    how='left'
)

In [54]:
df_merged

,Название проекта,Девелопер,Корпус,Срок сдачи,Стадия строительной готовности,Договор,id,ID ЖК,ID дом.рф
0,"16-я Парковая ул., д. 12",Фонд реновации,1,4 кв 2024,введен,ДКП,NaN,NaN,NaN
1,1864 Резиденция,Сбер Капитал,1,1 кв 2025,введен,ДКП,NaN,NaN,NaN
2,1-й Донской,ФСК,1,2 кв 2025,введен,ДКП,6921.0,NaN,NaN
3,1-й Донской,ФСК,2,2 кв 2025,введен,ДКП,6921.0,NaN,NaN
4,1-й Донской,ФСК,3,2 кв 2027,монтаж средних этажей,ДДУ,6921.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3120,Сберсити,СЗ Рублево-Архангельское,Квартал В17. Балансд. 2,1 кв 2026,монтаж последних этажей,ДДУ,5756.0,NaN,NaN
3121,Сберсити,СЗ Рублево-Архангельское,Квартал В11. Балансд. 2,4 кв 2025,благоустройство,ДДУ,5756.0,NaN,NaN
3122,ERA,Tekta,5,2 кв 2029,нулевой цикл,ДДУ,6763.0,NaN,NaN
3123,Ever,Tekta,1,1 кв 2026,благоустройство,ДДУ,5143.0,NaN,NaN


In [39]:
df_merged.to_excel(r"C:\Users\m.olshanskiy\Desktop\Что вышло.xlsx", index=False)

In [37]:
df_merged = df_merged.drop_duplicates()

Объединение по корпусу


In [69]:
df_baze = df_baze.merge(
    df_фильтр[['ID ЖК', 'Корпус', 'ID дом.рф']],
    left_on=['id', 'Корпус'],
    right_on=['ID ЖК', 'Корпус'],
    how='left'
)

In [70]:
df_baze

,Название проекта,Девелопер,Корпус,Срок сдачи,Стадия строительной готовности,Договор,id,ID ЖК,ID дом.рф
0,"16-я Парковая ул., д. 12",Фонд реновации,1,4 кв 2024,введен,ДКП,NaN,NaN,NaN
1,1864 Резиденция,Сбер Капитал,1,1 кв 2025,введен,ДКП,NaN,NaN,NaN
2,1-й Донской,ФСК,1,2 кв 2025,введен,ДКП,6921.0,6921.0,51450.0
3,1-й Донской,ФСК,2,2 кв 2025,введен,ДКП,6921.0,6921.0,51452.0
4,1-й Донской,ФСК,3,2 кв 2027,монтаж средних этажей,ДДУ,6921.0,6921.0,62732.0
...,...,...,...,...,...,...,...,...,...
2368,Сберсити,СЗ Рублево-Архангельское,Квартал В17. Балансд. 2,1 кв 2026,монтаж последних этажей,ДДУ,5756.0,NaN,NaN
2369,Сберсити,СЗ Рублево-Архангельское,Квартал В11. Балансд. 2,4 кв 2025,благоустройство,ДДУ,5756.0,NaN,NaN
2370,ERA,Tekta,5,2 кв 2029,нулевой цикл,ДДУ,6763.0,NaN,NaN
2371,Ever,Tekta,1,1 кв 2026,благоустройство,ДДУ,5143.0,5143.0,39040.0


In [72]:
df_baze.to_excel(r"C:\Users\m.olshanskiy\Desktop\Что вышло по корпусу.xlsx", index=False)

In [12]:
df_фильтр[df_фильтр['ID ЖК'] == 4298
]

,ID ЖК,ЖК рус,Тип помещения,Корпус,ID Корпус,lat,lng,ID дом.рф
803324,4298,Фили Сити,квартира,корпус 5 (Famous),11131.0,55.74365,37.51029,39481.0
803364,4298,Фили Сити,квартира,корпус 4 (Noble),10856.0,55.74365,37.51029,39481.0
803493,4298,Фили Сити,квартира,корпус 2 (Adagio),8448.0,55.74365,37.51029,10125.0
803695,4298,Фили Сити,квартира,корпус 3 (Vivo),8449.0,55.74365,37.51029,10126.0
803786,4298,Фили Сити,квартира,корпус 1 (Andante),8447.0,55.74365,37.51029,10125.0


In [11]:
df_фильтр[df_фильтр['ЖК рус'] == 'Эра'
]

,ID ЖК,ЖК рус,Тип помещения,Корпус,ID Корпус,lat,lng,ID дом.рф
880401,6763,Эра,квартира,корпус 3,17669.0,55.72032,37.645945,50651.0
880402,6763,Эра,квартира,корпус 1,17667.0,55.72032,37.645945,50651.0
880407,6763,Эра,квартира,корпус 2,17668.0,55.72032,37.645945,50651.0
880825,6763,Эра,квартира,корпус 4,19906.0,55.72032,37.645945,62125.0


In [14]:
result = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\Что вышло по корпусу.xlsx")

In [ ]:
result = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\Что вышло по корпусу.xlsx")

In [15]:
hashdom = pd.read_excel(r"C:\Users\m.olshanskiy\PycharmProjects\ndv_parsing\НашДомРФ\2025-11-19\НашДомРФ_глубже_МО.xlsx")

In [17]:
result = result.merge(
    hashdom[['id', 'Ввод в эксплуатацию']],
    left_on='ID дом.рф',
    right_on='id',
    how='left'
)


In [19]:
result = result.drop_duplicates()

In [20]:
result.to_excel(r"C:\Users\m.olshanskiy\Desktop\Что вышло по корпусу2.xlsx", index=False)